In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score
from scipy.optimize import linprog
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [24]:
merged_data=pd.read_csv('final_dataset.csv')

In [25]:
# Assuming your DataFrame is named 'df' (replace with the actual DataFrame variable)
columns = merged_data.columns.tolist()  # Convert to a list for better formatting
for col in columns:
    print(col)


state_name
district_name
crop_year
season
crop
area
production
dist code
year_x
state code
state name
dist name
adult male crossbred (1000 number)
adult male indigenous (1000 number)
adult female crossbred (1000 number)
adult female indigenous (1000 number)
young male crossbred (1000 number)
young male indigenous (1000 number)
young female crossbred (1000 number)
young female indigenous (1000 number)
st_code
state/region
actual rainfall
normal rainfall
dist.code
year_y
state.code
state.name
dist.name
rice.area..1000.ha.
rice.production..1000.tons.
rice.yield..kg.per.ha.
wheat.area..1000.ha.
wheat.production..1000.tons.
wheat.yield..kg.per.ha.
kharif.sorghum.area..1000.ha.
kharif.sorghum.production..1000.tons.
kharif.sorghum.yield..kg.per.ha.
rabi.sorghum.area..1000.ha.
rabi.sorghum.production..1000.tons.
rabi.sorghum.yield..kg.per.ha.
sorghum.area..1000.ha.
sorghum.production..1000.tons.
sorghum.yield..kg.per.ha.
pearl.millet.area..1000.ha.
pearl.millet.production..1000.tons.
pearl.mil

In [16]:
crop_production = pd.read_csv('crop_production.csv')
district_population = pd.read_csv('district_level_population.csv')
food = pd.read_csv('food.csv')
nutrition = pd.read_csv('nutrition.csv')
rainfall = pd.read_csv('rainfall.csv')
yield_complete = pd.read_csv('yield_complete.csv')
necessary_nutrition = pd.read_csv('necessary_nutrition.csv')

In [28]:
# Define age and gender groups and nutritional requirements
age_gender_groups = {
    'children_male': ['young male crossbred (1000 number)', 'young male indigenous (1000 number)'],
    'children_female': ['young female crossbred (1000 number)', 'young female indigenous (1000 number)'],
    'adults_male': ['adult male crossbred (1000 number)', 'adult male indigenous (1000 number)'],
    'adults_female': ['adult female crossbred (1000 number)', 'adult female indigenous (1000 number)']
}

required_nutrition = necessary_nutrition.set_index('nutrient')['daily_requirement'].to_dict()

# Calculate dietary needs based on population data
def calculate_dietary_needs(state, district):
    input_data = merged_data[(merged_data['state_name'].str.lower() == state.lower()) & (merged_data['district_name'].str.lower() == district.lower())]
    if input_data.empty:
        return "No data available for the given state and district."
    
    # Predict dietary needs based on population data
    dietary_needs = {}
    
    for nutrient, daily_requirement in required_nutrition.items():
        dietary_needs[nutrient] = {}
        for group, columns in age_gender_groups.items():
            group_population = sum(input_data[col].values[0] for col in columns) * 1000  # Convert thousands to actual number
            total_requirement = group_population * daily_requirement
            dietary_needs[nutrient][group] = total_requirement
    
    return dietary_needs

# Example usage
dietary_needs = calculate_dietary_needs('andhra pradesh', 'chittoor')
print(dietary_needs)


{'protein': {'children_male': 2122500.0, 'children_female': 17898000.0, 'adults_male': 1304500.0, 'adults_female': 26176500.0}, 'carbohydrates': {'children_male': 12735000.0, 'children_female': 107388000.0, 'adults_male': 7827000.0, 'adults_female': 157059000.0}, 'total fat': {'children_male': 2971500.0, 'children_female': 25057200.0, 'adults_male': 1826300.0, 'adults_female': 36647100.0}, 'saturated fat': {'children_male': 849000.0, 'children_female': 7159200.0, 'adults_male': 521800.0, 'adults_female': 10470600.0}, 'fiber': {'children_male': 1061250.0, 'children_female': 8949000.0, 'adults_male': 652250.0, 'adults_female': 13088250.0}, 'sugar': {'children_male': 3820500.0, 'children_female': 32216400.0, 'adults_male': 2348100.0, 'adults_female': 47117700.0}, 'vitamin a': {'children_male': 38205000.0, 'children_female': 322164000.0, 'adults_male': 23481000.0, 'adults_female': 471177000.0}, 'vitamin c': {'children_male': 3820500.0, 'children_female': 32216400.0, 'adults_male': 2348100.

In [35]:
def identify_surplus_deficit(state):
    state_data = merged_data[merged_data['state_name'].str.lower() == state.lower()]
    total_production = state_data.groupby('district_name')['production'].sum()
    dietary_needs = {}
    
    for district in state_data['district_name'].unique():
        dietary_needs[district] = calculate_dietary_needs(state, district)
    
    surplus_deficit = {}
    for district, needs in dietary_needs.items():
        production = total_production[district] * 1000  # Convert to actual number
        surplus_deficit[district] = {}
        
        for nutrient, group_needs in needs.items():
            total_need = sum(group_needs.values())
            surplus_deficit[district][nutrient] = production - total_need
    
    return surplus_deficit

# Example usage
surplus_deficit = identify_surplus_deficit('andhra pradesh')
print(surplus_deficit)


{'chittoor': {'protein': 374179956500.0, 'carbohydrates': 373942449000.0, 'total fat': 374160955900.0, 'saturated fat': 374208457400.0, 'fiber': 374203707250.0, 'sugar': 374141955300.0, 'vitamin a': 373372431000.0, 'vitamin c': 374141955300.0, 'calcium': 373277428000.0, 'iron': 374210357460.0, 'potassium': 369762317000.0, 'magnesium': 373847446000.0, 'zinc': 374217007670.0, 'vitamin d': 374208457400.0, 'vitamin b12': 374225177928.0}, 'east godavari': {'protein': 8993453741000.0, 'carbohydrates': 8993354736000.0, 'total fat': 8993445820600.0, 'saturated fat': 8993465621600.0, 'fiber': 8993463641500.0, 'sugar': 8993437900200.0, 'vitamin a': 8993117124000.0, 'vitamin c': 8993437900200.0, 'calcium': 8993077522000.0, 'iron': 8993466413640.0, 'potassium': 8991612248000.0, 'magnesium': 8993315134000.0, 'zinc': 8993469185780.0, 'vitamin d': 8993465621600.0, 'vitamin b12': 8993472591552.0}, 'guntur': {'protein': 86635449000.0, 'carbohydrates': 86603609000.0, 'total fat': 86632901800.0, 'saturat

In [47]:
def get_nutrition_info():
    nutrient_to_category = {
        'protein': 'protein',
        'carbohydrates': 'carbohydrate',
        'total fat': 'fat',
        'saturated fat': 'saturated fat',
        'fiber': 'fiber',
        'sugar': 'sugar',
        'vitamin a': 'vitamin a',
        'vitamin c': 'vitamin c',
        'calcium': 'calcium',
        'iron': 'iron',
        'potassium': 'potassium',
        'magnesium': 'magnesium',
        'zinc': 'zinc',
        'vitamin d': 'vitamin d',
        'vitamin b12': 'vitamin b12'
    }
    
    nutrition_info = {}
    for nutrient, category in nutrient_to_category.items():
        food_items = food[food['category'].str.contains(category, case=False, na=False)]
        description = necessary_nutrition[necessary_nutrition['nutrient'].str.lower() == nutrient.lower()]['description'].values[0]
        if not food_items.empty:
            nutrition_info[nutrient] = {
                'daily_requirement': required_nutrition[nutrient],
                'description': description,
                'food_items': food_items['description'].tolist()
            }
        else:
            nutrition_info[nutrient] = {
                'daily_requirement': required_nutrition[nutrient],
                'description': description,
                'food_items': ["No specific food items found"]
            }
    return nutrition_info

# Example usage
nutrition_info = get_nutrition_info()
print(nutrition_info)


{'protein': {'daily_requirement': 50.0, 'description': 'essential for growth and repair of tissues', 'food_items': ['textured vegetable protein', 'nutrition bar (snickers marathon protein bar)']}, 'carbohydrates': {'daily_requirement': 300.0, 'description': 'primary source of energy', 'food_items': ['No specific food items found']}, 'total fat': {'daily_requirement': 70.0, 'description': 'required for energy and cell function', 'food_items': ['No specific food items found']}, 'saturated fat': {'daily_requirement': 20.0, 'description': 'should be limited due to health concerns', 'food_items': ['No specific food items found']}, 'fiber': {'daily_requirement': 25.0, 'description': 'important for digestive health', 'food_items': ['cereal or granola bar (general mills fiber one chewy bar)', 'cereal (general mills fiber one)']}, 'sugar': {'daily_requirement': 90.0, 'description': 'should be limited to prevent health issues', 'food_items': ['cereal or granola bar (quaker chewy 25% less sugar g

In [43]:
# Sample structure of the food dataset
food = pd.DataFrame({
    'category': ['protein', 'fiber', 'sugar', 'protein', 'fiber'],
    'description': ['textured vegetable protein', 'cereal or granola bar (general mills fiber one chewy bar)', 
                    'cereal or granola bar (quaker chewy 25% less sugar granola bar)', 
                    'nutrition bar (snickers marathon protein bar)', 
                    'cereal (general mills fiber one)']
})

print(food.head())


  category                                        description
0  protein                         textured vegetable protein
1    fiber  cereal or granola bar (general mills fiber one...
2    sugar  cereal or granola bar (quaker chewy 25% less s...
3  protein      nutrition bar (snickers marathon protein bar)
4    fiber                   cereal (general mills fiber one)


In [48]:
from IPython.display import display, HTML

def display_results(state, district):
    dietary_needs = calculate_dietary_needs(state, district)
    surplus_deficit_status = identify_surplus_deficit(state)
    nutrition_info = get_nutrition_info()
    
    result = f"""
    <h3>Results for {state} - {district}</h3>
    <p><b>Dietary Needs:</b></p>
    <ul>
    """
    for nutrient, needs in dietary_needs.items():
        for group, value in needs.items():
            result += f"<li>{group} - {nutrient}: {value} mg</li>"
    result += f"""
    </ul>
    <p><b>Surplus/Deficit Status:</b></p>
    <ul>
    """
    for district, nutrients in surplus_deficit_status.items():
        result += f"<li>{district}: "
        for nutrient, value in nutrients.items():
            result += f"{nutrient}: {value} mg "
        result += "</li>"
    result += f"""
    </ul>
    <p><b>Nutritional Information:</b></p>
    <ul>
    """
    for nutrient, info in nutrition_info.items():
        result += f"<li>{nutrient}: {info['description']} (Daily Requirement: {info['daily_requirement']} mg) - Food items: {', '.join(info['food_items'])}</li>"
    result += f"""
    </ul>
    """
    display(HTML(result))

# Example usage
display_results('andhra pradesh', 'chittoor')
